In [1]:
import torch
import torch.nn as nn
import torchvision
import torch.utils.data as utils
from torchvision import transforms
import torchvision.models as models
from torch.utils.data import DataLoader, Dataset
import torch.optim as optim
from efficientnet_pytorch import EfficientNet

import cv2
import shutil
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

/home/umar_visionx/anaconda3/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [88]:
# !pip install --upgrade pip
# !pip install --upgrade efficientnet-pytorch

     |████████████████████████████████| 2.1 MB 305 kB/s            
  Attempting uninstall: pip
    Found existing installation: pip 21.3.1
    Uninstalling pip-21.3.1:
      Successfully uninstalled pip-21.3.1


In [2]:
import os
os.getcwd()

'/media/umar_visionx/Backup Plus/Active/Faizan'

In [3]:
train_dataset_path='/media/umar_visionx/Backup Plus/Active/Faizan/image_data/'
test_dataset_path='/media/umar_visionx/Backup Plus/Active/Faizan/test_images/'

In [4]:
# !mkdir test_images

In [5]:
#getting name of the classes
text_path='/media/umar_visionx/Backup Plus/Active/Faizan/doc_classifier_classes.txt'
with open(text_path,'r') as file:
    classes=file.read().strip().split('\n')

print(classes)

['Admin Note', 'Clinical History And Summary', 'Consult Note', 'Cover Page', 'Imaging Note', 'Insurance Authorization', 'Intake Forms', 'Lab Test', 'Other', 'Patient Profile', 'Prescriptions', 'Referral Letter', 'Requisition Form']


In [6]:
# #making classes folder in test_data folder
# for name in classes:
#     path=os.path.join(test_dataset_path,name)
#     os.makedirs(path)
# print('process completed')

In [7]:
# #copying test data to new folder
# test_path='/media/umar_visionx/Backup Plus/Active/Faizan/data/test-set/Active'
# for hosp in os.listdir(test_path):
#     hosp_path=os.path.join(test_path,hosp)
#     for cls in os.listdir(hosp_path):
#         if cls in classes:#discarding EOB
#             cls_path=os.path.join(hosp_path,cls)
#             for folder in os.listdir(cls_path):
#                 folder_path=os.path.join(cls_path,folder)
#                 for file in os.listdir(folder_path):
#                     if file.endswith('.png'):
#                         img_path=os.path.join(folder_path,file)
#                         dest_path=os.path.join(test_dataset_path,cls)
#                         shutil.copy(img_path,dest_path)
                        
#                     else:
#                         pass
#         else:
#             pass
# print("copying process completed")

In [8]:
#label encoding
labl=[]
for folder in os.listdir(train_dataset_path):
  labl.append(folder)#making the list of folder names which are labels


#encoding the labels from the list
label_index={}
for index,label in enumerate(labl):
  label_index[label]=index

label_index#label to index

{'Admin Note': 0,
 'Clinical History And Summary': 1,
 'Consult Note': 2,
 'Cover Page': 3,
 'Imaging Note': 4,
 'Insurance Authorization': 5,
 'Intake Forms': 6,
 'Lab Test': 7,
 'Other': 8,
 'Patient Profile': 9,
 'Prescriptions': 10,
 'Referral Letter': 11,
 'Requisition Form': 12}

In [9]:
#making dictionary for label decoding
index_label={}
for key,val in zip(label_index.keys(),label_index.values()):
    index_label[val]=key
print(index_label)

{0: 'Admin Note', 1: 'Clinical History And Summary', 2: 'Consult Note', 3: 'Cover Page', 4: 'Imaging Note', 5: 'Insurance Authorization', 6: 'Intake Forms', 7: 'Lab Test', 8: 'Other', 9: 'Patient Profile', 10: 'Prescriptions', 11: 'Referral Letter', 12: 'Requisition Form'}


In [10]:
def preprocessing(path):
  #getting images path and labels
  targets=[]
  images=[]
  labels=[]
  #images_labels=[]
  #imges_labels={}
  for folder in os.listdir(path):
    folder_path=path + folder
    for img in os.listdir(folder_path):
      img_path=folder_path + '/'+ img
      targets.append(folder)
      images.append(img_path)

  #encoding all labels
  for ent in targets:
    labels.append(label_index[ent])

  #converting into datadrame
  dframe= pd.DataFrame()
  dframe["image_path"]=images
  dframe["label"]=labels
  return dframe

In [58]:
df_tr=preprocessing(train_dataset_path)
df_ts=preprocessing(test_dataset_path)
print(df_tr.head())
df_ts.head()

                                          image_path  label
0  /media/umar_visionx/Backup Plus/Active/Faizan/...      0
1  /media/umar_visionx/Backup Plus/Active/Faizan/...      0
2  /media/umar_visionx/Backup Plus/Active/Faizan/...      0
3  /media/umar_visionx/Backup Plus/Active/Faizan/...      0
4  /media/umar_visionx/Backup Plus/Active/Faizan/...      0


,image_path,label
0,/media/umar_visionx/Backup Plus/Active/Faizan/...,0
1,/media/umar_visionx/Backup Plus/Active/Faizan/...,0
2,/media/umar_visionx/Backup Plus/Active/Faizan/...,0
3,/media/umar_visionx/Backup Plus/Active/Faizan/...,0
4,/media/umar_visionx/Backup Plus/Active/Faizan/...,0


In [59]:
print(len(df_tr))
print(len(df_ts))

11660
1810


In [25]:
# path='/media/umar_visionx/Backup Plus/Active/Faizan/test_data'
# total=[]
# for fold in os.listdir(path):
#     fold_path=os.path.join(path,fold)
#     for img in os.listdir(fold_path):
#         img_path=os.path.join(fold_path,img)
#         total.append(img_path)
# print(len(total))


11664


In [11]:
#csv path
csv_path='/media/umar_visionx/Backup Plus/Active/Faizan/Visul&TextualFeature_csv'

In [61]:
##saving the train and test dataframes
#df_tr.to_csv(os.path.join(csv_path,'train_data.csv'),index = False, header=True)
#df_ts.to_csv(os.path.join(csv_path,'test_data.csv'),index = False, header=True)

In [12]:
#loading dataframe
df_train=pd.read_csv(os.path.join(csv_path,'train_data.csv'))
df_test=pd.read_csv(os.path.join(csv_path,'test_data.csv'))
print(df_train.head())
df_test.head()

                                          image_path  label
0  /media/umar_visionx/Backup Plus/Active/Faizan/...      0
1  /media/umar_visionx/Backup Plus/Active/Faizan/...      0
2  /media/umar_visionx/Backup Plus/Active/Faizan/...      0
3  /media/umar_visionx/Backup Plus/Active/Faizan/...      0
4  /media/umar_visionx/Backup Plus/Active/Faizan/...      0


,image_path,label
0,/media/umar_visionx/Backup Plus/Active/Faizan/...,0
1,/media/umar_visionx/Backup Plus/Active/Faizan/...,0
2,/media/umar_visionx/Backup Plus/Active/Faizan/...,0
3,/media/umar_visionx/Backup Plus/Active/Faizan/...,0
4,/media/umar_visionx/Backup Plus/Active/Faizan/...,0


In [13]:
transform =  transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize(size=(224,224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[123.675, 116.28, 103.53], 
                             std=[58.395, 57.12, 57.375])])

In [14]:
class PhelixDataset(Dataset):
    def __init__(self, df):
        #super().__init__()
        self.df = df
        self.images = self.df['image_path']
        self.labels =self.df['label']

    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, index):       
        img_path = self.images[index]
        label = self.labels[index]
        
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = transform(image)
        return image,label

In [152]:
#from PIL import Image
# img_path = df_train['image_path'][1]
# label = df_train['label'][1]
# image = cv2.imread(img_path)
# image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
# image
# #image.shape
# # #im_pil = Image.fromarray(image)
# # image = transform(image)
# # image
# # #type(image)

<class 'str'>
<class 'numpy.int64'>


In [15]:
batch_size=32
train_data = PhelixDataset(df_train)
test_data = PhelixDataset(df_test)
train_dataloader = DataLoader(train_data, batch_size = batch_size,shuffle=True,num_workers=3,pin_memory=True)
test_dataloader = DataLoader(test_data, batch_size = batch_size,shuffle=False,num_workers=3,pin_memory=True)
sample = next(iter(train_dataloader))
print(sample)
# batch=next(iter(train_dataloader))
# print(batch)

[tensor([[[[-2.1008, -2.1008, -2.1008,  ..., -2.1008, -2.1008, -2.1008],
          [-2.1008, -2.1008, -2.1008,  ..., -2.1008, -2.1008, -2.1008],
          [-2.1008, -2.1008, -2.1008,  ..., -2.1008, -2.1008, -2.1008],
          ...,
          [-2.1008, -2.1008, -2.1008,  ..., -2.1008, -2.1008, -2.1008],
          [-2.1008, -2.1008, -2.1008,  ..., -2.1008, -2.1008, -2.1008],
          [-2.1008, -2.1008, -2.1008,  ..., -2.1008, -2.1008, -2.1008]],

         [[-2.0182, -2.0182, -2.0182,  ..., -2.0182, -2.0182, -2.0182],
          [-2.0182, -2.0182, -2.0182,  ..., -2.0182, -2.0182, -2.0182],
          [-2.0182, -2.0182, -2.0182,  ..., -2.0182, -2.0182, -2.0182],
          ...,
          [-2.0182, -2.0182, -2.0182,  ..., -2.0182, -2.0182, -2.0182],
          [-2.0182, -2.0182, -2.0182,  ..., -2.0182, -2.0182, -2.0182],
          [-2.0182, -2.0182, -2.0182,  ..., -2.0182, -2.0182, -2.0182]],

         [[-1.7870, -1.7870, -1.7870,  ..., -1.7870, -1.7870, -1.7870],
          [-1.7870, -1.7870, 

In [16]:
for img, label in train_dataloader:
    print(img.shape)
    print(label.shape)
    break

torch.Size([32, 3, 224, 224])
torch.Size([32])


In [18]:
#efficientnet_b0 = models.efficientnet_b0()
#model = EfficientNet.from_name('efficientnet-b0')
model = models.efficientnet_b0(pretrained=True)

In [19]:
# Freezing the model layers
for parameter in model.parameters():
    parameter.requires_grad = False

In [26]:
#changing the classifier layer
model.classifier = nn.Sequential(nn.Flatten(),
                           nn.Linear(1280,512),
                           nn.ReLU(),
                           nn.Dropout(p=0.4),
                           nn.Linear(512,128),
                           nn.ReLU(),
                           nn.Dropout(p=0.4),
                           nn.Linear(128,13))#totel classes are 13

In [27]:
# making Classifier layer Trainable                           
for parameter in model.classifier.parameters():
  parameter.requires_grad = True

In [28]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

EfficientNet(
  (features): Sequential(
    (0): ConvNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): MBConv(
        (block): Sequential(
          (0): ConvNormActivation(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (avgpool): AdaptiveAvgPool2d(output_size=1)
            (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
            (fc2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
            (activation): SiLU(inplace=True)
            (scale_activation): Sigmoid()
          )
          (2): ConvNormActivation(
 

In [29]:
num_epochs = 30
learning_rate = 0.0001
batch_size = 32

In [30]:
optimizer = optim.Adam(model.classifier.parameters(), lr=learning_rate)
criterion=nn.CrossEntropyLoss()

In [ ]:
model_chkp_path="/media/umar_visionx/Backup Plus/Active/Faizan/model_chkpt/model1.pt"

In [33]:
#training the model
global_step = 0

#put the model in training mode
model.train()
###
train_epoch=[]
train_acc=[]
val_acc=[]
curr_loss=[]
#val_epoch=[]
#train_losses, valid_losses,valid_acc = [], [], []


for epoch in range(num_epochs):
  print("Epoch:", epoch)
  ###
  train_epoch.append(epoch)
  running_loss = 0.0
  correct = 0
  for images, labels in train_dataloader:
   
      #moving to device
      images = images.to(device)
      labels = labels.to(device)

  #     # forward pass
      logits = model(images)
      loss = criterion(logits,labels)
    #   print(logits)
    #   print(labels)
      
  
      running_loss += loss.item()
      predictions = logits.argmax(-1)
      correct += (predictions == labels).float().sum()
    #   print(loss)
  #     # backward pass to get the gradients 
      loss.backward()

  #     # update
      optimizer.step()
      optimizer.zero_grad()
      global_step += 1

  
  print("Loss:", running_loss / batch_size)
  accuracy = 100 * correct / len(df_train)
  print("Training accuracy:", accuracy.item())
  ###
  train_acc.append(accuracy.item())
  ###
  curr_loss.append(running_loss / batch_size)
  #saving model
  torch.save(model.state_dict(), model_chkp_path)

#   #checking on validation dataset
#   #if epoch%5==0:
#   model.eval()

#   correct = 0
#   for batch in val_dataloader:

#     #moving to device
#     images = images.to(device)
#     labels = labels.to(device)


#     logits = model(images)

#     predictions = logits.argmax(-1)
#     correct += (predictions == labels).float().sum()

#   accuracy = 100 * correct / len(df_val)
#   print("Validation accuracy:", accuracy.item())
#     ###
#   val_acc.append(accuracy.item())
    
  print("-----------")

Epoch: 0
Loss: 16.236709635704756
Training accuracy: 55.42023849487305
-----------
Epoch: 1
Loss: 14.326438020914793
Training accuracy: 60.31732177734375
-----------
Epoch: 2
Loss: 13.135047318413854
Training accuracy: 63.35334396362305
-----------
Epoch: 3
Loss: 12.591167248785496
Training accuracy: 65.02572631835938
-----------
Epoch: 4
Loss: 11.800845459103584
Training accuracy: 67.08404541015625
-----------
Epoch: 5
Loss: 11.252525808289647
Training accuracy: 68.7392807006836
-----------
Epoch: 6
Loss: 10.761295653879642
Training accuracy: 70.0343017578125
-----------
Epoch: 7
Loss: 10.416289787739515
Training accuracy: 71.11492156982422
-----------
Epoch: 8
Loss: 10.236621338874102
Training accuracy: 71.42366790771484
-----------
Epoch: 9


In [ ]:
#creating a dataframe from lists to plot the graphs
df_an = pd.DataFrame(list(zip(train_epoch, train_acc,curr_loss,val_acc)),
               columns =['epoch', 'train_acc','loss','val_acc'])
df_an.head()

In [ ]:
#plotting training epochs,loss,accuracy and validation accuracy
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

plt.figure(figsize=(8, 6), dpi=80)
sns.lineplot(data = df_an)
plt.xlabel("epochs")
plt.ylabel("value")
plt.title("loss,train_accuracy,valid_accuracy vrs epochs ")
plt.plot()

In [ ]:
#loading the model
model.load_state_dict(torch.load(model_chkp_path))
model.eval()

In [ ]:
#checking model performance on testset
model.eval()
labels_output=[]
predictions_output=[]

###
y_pred=[]
y_label=[]
correct = 0
for images,labels in test_dataloader:
    
    #moving to device
    images = images.to(device)
    labels = labels.to(device)
  
    with torch.no_grad():
      logits = model(images)


    predictions = logits.argmax(-1)
    ###
    y_pred.append(predictions)
    ###
    y_label.append(labels)
    #print(labels)
    #print(predictions)
    correct += (predictions == labels).float().sum()
    #print(correct)

accuracy = 100 * correct / len(df_test)
print("Testing accuracy:", accuracy.item())

In [ ]:
#extracting the labels from from tensors and making a list
label0=[]
label1=[]
labels=[] #extracted label

for val0 in y_label:
  #converted to numpy array
  label0.append(val0.cpu().numpy())
for val2 in label0:
  label1.append(val2.tolist())#converting from array to list of list of labels
for val3 in label1:#converting in list of labels
  for val4 in val3:
    labels.append(val4)

labels[0:5]

In [ ]:
#extracting the predictions from tensors and making a list
pred0=[]
pred1=[]
predictions=[] #extracted label

for val0 in y_pred:
  #converted to numpy array
  pred0.append(val0.cpu().numpy())
for val2 in pred0:
  pred1.append(val2.tolist())#converting from array to list of list of labels
for val3 in pred1:#converting in list of labels
  for val4 in val3:
    predictions.append(val4)

predictions[0:5]

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
#confusion matrix
cm = confusion_matrix(labels, predictions)
cm

In [ ]:
# Creating a dataframe for a array-formatted Confusion matrix,so it will be easy for plotting.
cm_df = pd.DataFrame(cm,
                     index = [['Clinical History And Summary','Lab Test','Requisition Form','Other','Admin Note','Patient Profile','Consult Note','Cover Page']], 
                     columns = [['Clinical History And Summary','Lab Test','Requisition Form','Other','Admin Note','Patient Profile','Consult Note','Cover Page']])
cm_df

In [ ]:
#Plotting the confusion matrix
plt.figure(figsize=(5,4))
sns.heatmap(cm_df, annot=True)
plt.title('Confusion Matrix')
plt.ylabel('Actual Values')
plt.xlabel('Predicted Values')
plt.show()

In [ ]:
#classification report
target_names = ['Clinical History And Summary','Lab Test','Requisition Form','Other','Admin Note','Patient Profile','Consult Note','Cover Page']
print(classification_report(labels, predictions, target_names=target_names))

In [ ]:
from sklearn import metrics
print(f"Accuracy Score: {(metrics.accuracy_score(labels, predictions)) * 100} %" )